# FloPy 

### Working stack demo

A short demonstration of some of the `flopy3` awesomeness

In [ ]:
%matplotlib inline
from IPython.display import clear_output, display
from __future__ import print_function
import time
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import flopy

###Model Inputs

In [ ]:
# first lets load an existing model
model_ws = os.path.join("..","data","freyberg_multilayer_transient")
ml = flopy.modflow.Modflow.load("freyberg.nam", model_ws=model_ws, verbose=False,
                               check=False,exe_name="mfnwt")

In [ ]:
ml.dis.sr

Let's looks at some plots

In [ ]:
ml.upw.plot()

In [ ]:
ml.dis.plot()

In [ ]:
ml.drn.plot(key="cond")
ml.drn.plot(key="elev")


Let's write a shapefile of the DIS package

In [ ]:
ml.dis.export(os.path.join(model_ws,"freyberg_dis.shp"))

Let's also write a netCDF file with all model inputs

In [ ]:
ml.export(os.path.join(model_ws, "freyberg.nc"))

###Change model directory, modify inputs and write new input files

In [ ]:
ml.external_path = "ref"
ml.model_ws = "temp"
#print(ml.upw.hk[0].array.mean())
#ml.upw.hk[0] *= 2
#print(ml.upw.hk[0].array.mean())


### Run the model

In [ ]:
ml.write_input()

ml.run_model()

### Output inspection

First, let's look at the list file

In [ ]:
mfl = flopy.utils.MfListBudget(os.path.join(model_ws,"freyberg.list"))
df_flux, df_vol = mfl.get_dataframes(start_datetime="10-21-2015")
df_flux

In [ ]:
groups = df_flux.groupby(lambda x:x.split('_')[-1],axis=1).groups
df_flux_in = df_flux.loc[:,groups["IN"]]
df_flux_in.columns = df_flux_in.columns.map(lambda x:x.split('_')[0])

df_flux_out = df_flux.loc[:,groups["OUT"]]
df_flux_out.columns = df_flux_out.columns.map(lambda x:x.split('_')[0])


df_flux_delta = df_flux_in - df_flux_out
#df_flux_delta = df_flux.loc[:,"in"] - df_flux.loc[:,"out"]
df_flux_delta.iloc[-1,:].plot(kind="bar",figsize=(10,10),grid=True)

Let's look at head

In [ ]:
# if you pass the model instance, then the plots will be offset and rotated
h = flopy.utils.HeadFile(os.path.join(model_ws,"freyberg.hds"),model=ml)
h.times

In [ ]:
h.plot(totim=900,contour=True,grid=True,colorbar=True,figsize=(10,10))

In [ ]:
#let's write these heads to shapefile
h.to_shapefile(os.path.join(model_ws,"freyburg_head.shp"))

### Let's make an outputs movie

In [ ]:
f = plt.figure(figsize=(10,10))
ax = plt.subplot(1,1,1,aspect="equal")
for i,t in enumerate(h.times):
    ax.set_title("totim:{0}".format(t))
    h.plot(totim=t,mflay=0,contour=True,
           grid=True,figsize=(10,10),
            vmin=0,vmax=20,axes=[ax])
    time.sleep(0.01)
    clear_output(True)
    display(f)
    ax.cla()